<a href="https://colab.research.google.com/github/ArmaanSethi/Gemini-Soccer-Scout/blob/main/AI_Soccer_Scout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import requests
import pandas as pd

from datetime import datetime
from bs4 import BeautifulSoup
from google.colab import userdata



In [56]:
# Lamine Yamal
player_name = 'Lamine Yamal'
url = 'https://fbref.com/en/players/82ec26c1/Lamine-Yamal'
attrs = 'scout_summary_AM'


In [57]:
df = pd.read_html(
    url,
    attrs={'id': attrs} # Change the id & class to correspond to the table
)[0]

In [58]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')


In [59]:
# Get the player's age, team, league, and position
position = soup.select_one('p:-soup-contains("Position")').text.split(':')[-2].split(' ')[0].strip()
birthday = soup.select_one('span[id="necro-birth"]').text.strip()

# get the player's age
age = (datetime.now() - datetime.strptime(birthday, '%B %d, %Y')).days // 365
team = soup.select_one('p:-soup-contains("Club")').text.split(':')[-1].strip()


In [62]:
df = df.dropna(subset='Statistic')
df

,Statistic,Per 90,Percentile
0,Non-Penalty Goals,0.15,24.0
1,npxG: Non-Penalty xG,0.24,58.0
2,Shots Total,2.61,71.0
3,Assists,0.22,63.0
4,xAG: Exp. Assisted Goals,0.28,83.0
5,npxG + xAG,0.51,74.0
6,Shot-Creating Actions,4.49,71.0
8,Passes Attempted,44.58,71.0
9,Pass Completion %,78.6%,72.0
10,Progressive Passes,3.38,36.0


In [63]:
prompt = f"""

I need you to create a scouting report on {player_name}. Can you provide me with a summary of their strengths and weaknesses?

Here is the data I have on him:

Player: {player_name}
Position: {position}
Age: {age}
Team: {team}

{df.to_markdown()}


Return the scouting report in the following markdown format:

# Scouting Report for {player_name}

## Strengths
< a list of 1 to 3 strengths >

## Weaknesses
< a list of 1 to 3 weaknesses >

## Summary
< a brief summary of the player's overall performance and if he would be beneficial to the team >
"""
prompt

"\n\nI need you to create a scouting report on Lamine Yamal. Can you provide me with a summary of their strengths and weaknesses?\n\nHere is the data I have on him:\n\nPlayer: Lamine Yamal\nPosition: \nAge: 16\nTeam: Barcelona\n\n|    | Statistic                | Per 90   |   Percentile |\n|---:|:-------------------------|:---------|-------------:|\n|  0 | Non-Penalty Goals        | 0.15     |           24 |\n|  1 | npxG: Non-Penalty xG     | 0.24     |           58 |\n|  2 | Shots Total              | 2.61     |           71 |\n|  3 | Assists                  | 0.22     |           63 |\n|  4 | xAG: Exp. Assisted Goals | 0.28     |           83 |\n|  5 | npxG + xAG               | 0.51     |           74 |\n|  6 | Shot-Creating Actions    | 4.49     |           71 |\n|  8 | Passes Attempted         | 44.58    |           71 |\n|  9 | Pass Completion %        | 78.6%    |           72 |\n| 10 | Progressive Passes       | 3.38     |           36 |\n| 11 | Progressive Carries      | 4.97

In [64]:
print(prompt)



I need you to create a scouting report on Lamine Yamal. Can you provide me with a summary of their strengths and weaknesses?

Here is the data I have on him:

Player: Lamine Yamal
Position: 
Age: 16
Team: Barcelona

|    | Statistic                | Per 90   |   Percentile |
|---:|:-------------------------|:---------|-------------:|
|  0 | Non-Penalty Goals        | 0.15     |           24 |
|  1 | npxG: Non-Penalty xG     | 0.24     |           58 |
|  2 | Shots Total              | 2.61     |           71 |
|  3 | Assists                  | 0.22     |           63 |
|  4 | xAG: Exp. Assisted Goals | 0.28     |           83 |
|  5 | npxG + xAG               | 0.51     |           74 |
|  6 | Shot-Creating Actions    | 4.49     |           71 |
|  8 | Passes Attempted         | 44.58    |           71 |
|  9 | Pass Completion %        | 78.6%    |           72 |
| 10 | Progressive Passes       | 3.38     |           36 |
| 11 | Progressive Carries      | 4.97     |           80 |
| 

In [65]:
API_KEY=userdata.get('API_KEY')

In [66]:
"""
At the command line, only need to run once to install the package via pip:

$ pip install google-generativeai
"""

import google.generativeai as genai
genai.configure(api_key=API_KEY)

# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]



In [67]:
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

convo = model.start_chat(history=[
])

In [68]:
convo.send_message(prompt)
# print(convo.last.text)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': "# Scouting Report for Lamine Yamal\n\n## Strengths\n\n*   **Dribbling and Chance Creation:** Yamal excels at taking on defenders and creating scoring opportunities for himself and his teammates. His high percentile rankings in successful take-ons, shot-creating actions, and xAG demonstrate his ability to be a dynamic threat in the final third. \n*   **Ball Retention and Passing:** He displays good passing ability with a high pass completion rate, indicating his ability to keep possession and distribute the ball effectively.  \n*   **Defensive Contribution:** Yamal's defensive stats are impressive for an attacking player. His high percentile rankings in tackles, interceptions, and blocks show his willingness to contribute defensively and press opponents. \n\n## Weaknesses\n\n*   **Goal Scoring:** While his xG suggests he gets into g

In [69]:
from pathlib import Path
from IPython.display import Markdown


In [70]:
display(Markdown(convo.last.text))

# Scouting Report for Lamine Yamal

## Strengths

*   **Dribbling and Chance Creation:** Yamal excels at taking on defenders and creating scoring opportunities for himself and his teammates. His high percentile rankings in successful take-ons, shot-creating actions, and xAG demonstrate his ability to be a dynamic threat in the final third. 
*   **Ball Retention and Passing:** He displays good passing ability with a high pass completion rate, indicating his ability to keep possession and distribute the ball effectively.  
*   **Defensive Contribution:** Yamal's defensive stats are impressive for an attacking player. His high percentile rankings in tackles, interceptions, and blocks show his willingness to contribute defensively and press opponents. 

## Weaknesses

*   **Goal Scoring:** While his xG suggests he gets into good scoring positions, his actual goal-scoring output is relatively low. Improving his finishing ability will be crucial for him to become a more well-rounded attacker.
*   **Aerial Ability:** Yamal's aerial duel success rate is very low, which could be a limitation in certain situations. 

## Summary

Lamine Yamal is a talented young player with exciting potential. His dribbling skills, creativity, and defensive contributions make him a valuable asset to any team. While his goal-scoring could improve, his ability to create chances for himself and others suggests that he could develop into a top-class attacker with time and experience. He would likely be a beneficial addition to many teams, especially those that value dynamic wingers who can contribute both offensively and defensively. 


In [71]:
# Print to file.
file_path = Path(f"{player_name}_scouting_report.md")
file_path.write_text(convo.last.text)

1631

# Future Plans
### Include Football Manager stats from internet
* https://fminside.net/players/4-fm-24/2000256231-lamine-yamal
### Include video understanding with Gemini 1.5 pro
* https://github.com/google-gemini/cookbook/blob/main/quickstarts/Video.ipynb
